In [ ]:
# SO THAT IT CAN BE RE-RUN
%load_ext autoreload
%autoreload 2

# Define fitness functions to be used by  the integration algorithms 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import minimize



In [ ]:
#all fitness functions
def EOSA_fitness(agent, trainX, testX, trainy, testy):
    cols=np.flatnonzero(agent)
    #val=1
    #if np.shape(cols)[0]==0:
     #   return val, 1-val
    clf=KNeighborsClassifier(n_neighbors=5)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=1-clf.score(test_data,testy)
    
    set_cnt=sum(agent)
    set_cnt=set_cnt/np.shape(agent)[0]
    val=omega*val+(1-omega)*set_cnt
    return val, 1-val

def fitness(agent, trainX, testX, trainy, testy):
    cols=np.flatnonzero(agent)
    #val=1
    #if np.shape(cols)[0]==0:
     #   return val, 1-val
    
    clf=KNeighborsClassifier(n_neighbors=5)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=1-clf.score(test_data,testy)
    set_cnt=sum(agent)
    set_cnt=set_cnt/np.shape(agent)[0]
    val=omega*val+(1-omega)*set_cnt
    return val, 1-val

def BDMO_fitness(agent, trainX, testX, trainy, testy):
    cols=np.flatnonzero(agent)
    #val=1
    #if np.shape(cols)[0]==0:
    #    return val, 1-val
    clf=KNeighborsClassifier(n_neighbors=5)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=1-clf.score(test_data,testy)
    set_cnt=sum(agent)
    set_cnt=set_cnt/np.shape(agent)[0]
    val=omega*val+(1-omega)*set_cnt
    return val, 1-val

def BDMO_allfit(pop, trainX, testX, trainy, testy):
    acc = np.zeros((len(pop), 1))
    cost = np.zeros((len(pop), 1))
    for p in range(len(pop)):
        acc[p], cost[p] = BDMO_fitness(pop[p], trainX, testX, trainy, testy)
    return acc, cost

def allfit(pop, trainX, testX, trainy, testy):
    fit = np.zeros((len(pop), 1))
    cost = np.zeros((len(pop), 1))
    for p in range(len(pop)):
        fit[p], cost[p] = fitness(pop[p], trainX, testX, trainy, testy)
    return fit, cost

def get_global_best(pop, id_fitness, id_best):
    minn = 100
    temp = pop[0]
    for i in pop:
        #print(i)
        #print(i[1])
        if isinstance(i[1], tuple):
            fit, cost=i[1]
        else:
            fit=i[1]
        minn = min(minn, fit)
        temp = i
    return temp

def sort_using_fitness(pops):
    ID_POP=0
    ID_ACC_COST=1
    ID_ACC=0
    ID_COST=1
    acc, cost, pop=[], [], []
    sorted_pop = sorted(pops, key=lambda agent: agent[ID_ACC_COST][ID_ACC])  
    for p in range(len(sorted_pop)):
        pop.append(sorted_pop[p][ID_POP])
        acc.append(sorted_pop[p][ID_ACC_COST][ID_ACC])
        cost.append(sorted_pop[p][ID_ACC_COST][ID_COST])
    return acc, cost, pop

def EOSA_allfit(pop, trainX, testX, trainy, testy):
    acc = []
    cost = []
    pops=[]
    for p in range(len(pop)):
        # print('about to see what is in pop[p]')
        # print(pop[p])
        
        idx, ind=pop[p]
        indX=ind[EOSA_ID_POS]
        # print(indX)
        ac, ct = EOSA_fitness(indX, trainX, testX, trainy, testy)
        acc.append(ac)
        cost.append(ct)
        ind[EOSA_ID_FIT]=ac #the actual fitness values
        pop[p]=idx, ind
        pops.append([pop[p], [ac, ct]])
    return sort_using_fitness(pops)

# Independent Binary snake optimization algorithm

In [ ]:

# integration algorithms as fitness functions 

def ACO(agents, fitAgent, trainX, testX, trainy, testy, dim, isNormalPop, curIter, MaxIter):
    popSize = len(agents)
    reset_ACO()
    
    # ACO Parameters
    ALPHA = 1.0      # Pheromone importance
    BETA = 2.0       # Heuristic importance
    RHO = 0.5        # Evaporation rate
    Q = 100          # Pheromone constant
    TAU_MIN = 0.01   # Minimum pheromone
    TAU_MAX = 6.0    # Maximum pheromone
    
    # Initialize pheromone matrix if this is first iteration
    if not hasattr(ACO, 'pheromone'):
        ACO.pheromone = np.full(dim, 0.5)  # Initial pheromone levels
    
    # Initialize best solution tracking
    if not hasattr(ACO, 'best_solution'):
        ACO.best_solution = None
        ACO.best_fitness = float('inf')
    
    # Get fitness values
    if isNormalPop:
        fitList = allfit(agents, trainX, trainy, testX, testy)
    else:
        fitList = []
        for agent in agents:
            fitList.append(agent[EOSA_INDIVIDUAL][EOSA_ID_FIT])
    
    # Update global best solution
    for i in range(popSize):
        current_fitness = fitList[i]
        if current_fitness < ACO.best_fitness:
            ACO.best_fitness = current_fitness
            if isNormalPop:
                ACO.best_solution = agents[i].copy()
            else:
                ACO.best_solution = agents[i][EOSA_INDIVIDUAL][EOSA_ID_POS].copy()
    
    # Generate new population of ants
    new_agents = []
    
    for ant_idx in range(popSize):
        # Construct solution using pheromone-based probabilistic selection
        new_solution = construct_ant_solution(ACO.pheromone, dim, ALPHA, BETA)
        
        if isNormalPop:
            new_agents.append(new_solution)
        else:
            # Calculate fitness for new solution
            fit, cost = EOSA_fitness(new_solution, trainX, testX, trainy, testy)
            index, _ = agents[ant_idx]
            individual = index, [new_solution, fit]
            new_agents.append(individual)
    
    # Update pheromone trails
    update_pheromone(ACO.pheromone, agents, fitList, isNormalPop, 
                    RHO, Q, TAU_MIN, TAU_MAX, dim)
    
    return new_agents


In [ ]:
import numpy as np
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

class BinarySnakeOptimizer:
    """
    Binary Snake Optimizer (BSO) for Feature Selection
    
    Implementation based on: "An improved binary snake optimizer with Gaussian mutation 
    transfer function and hamming distance for feature selection" by Bao et al. (2024)
    """
    
    def __init__(self, n_snakes=30, max_iterations=100, alpha=0.99, accuracy_func=None):
        """
        Initialize Binary Snake Optimizer
        
        Parameters from Section 6.2 (Table 3):
        - n_snakes: Population size (set to 30 as per experimental setup)
        - max_iterations: Maximum iterations (set to 100 as per experimental setup)
        - alpha: Weight parameter in fitness function (Equation 14, set to 0.99)
        - accuracy_func: Custom accuracy evaluation function
        
        Constants from Section 3 (Snake Optimization Algorithm):
        - c1: Constant equal to 0.5 (Equation 5)
        - c2: Constant equal to 0.05 (Equation 6) 
        - c3: Constant equal to 2 (Equation 8)
        - threshold_temp: Temperature threshold = 0.6 (Section 6.2)
        - threshold_food: Food threshold = 0.25 (Section 6.2)
        """
        self.n_snakes = n_snakes
        self.max_iterations = max_iterations
        self.alpha = alpha  # Weight parameter from Equation 14
        self.accuracy_func = accuracy_func or self._default_accuracy_func
        
        # Algorithm constants from Section 3 and Table 3
        self.c1 = 0.5      # Equation 5: Q = c1 * exp((t-T)/T)
        self.c2 = 0.05     # Equation 6: exploration phase constant
        self.c3 = 2        # Equation 8: exploitation phase constant
        self.threshold_temp = 0.6   # Temperature threshold (Section 6.2)
        self.threshold_food = 0.25  # Food threshold (Section 6.2)
        
    def _default_accuracy_func(self, X_train, y_train, X_test, y_test, selected_features):
        """
        Default accuracy function using KNN classifier
        
        From Section 4: "choosing classifiers with lower computational cost, 
        such as K-nearest neighbors (KNN), can help mitigate this problem"
        
        From Section 6.3: "we utilize the KNN (k=5) as the classifier"
        """
        if np.sum(selected_features) == 0:
            return 0.0

        selected_features = np.where(selected_features)[0]
            
        X_train_selected = X_train[:, selected_features]
        X_test_selected = X_test[:, selected_features]
        
        knn = KNeighborsClassifier(n_neighbors=5)  # k=5 as per Section 6.3
        knn.fit(X_train_selected, y_train)
        y_pred = knn.predict(X_test_selected)
        
        return accuracy_score(y_test, y_pred)
    
    def _fitness_function(self, X_train, y_train, X_test, y_test, selected_features):
        """
        Fitness function based on linear weighted method
        
        From Section 4, Equation 14:
        Fitness = α * (1 - acc) + (1 - α) * (F_s / F_all)
        
        where:
        - acc: prediction accuracy of machine learning classifier
        - F_s: number of selected features
        - F_all: total number of features
        - α: weight parameter (set to 0.99 in Section 6.3)
        """
        acc = self.accuracy_func(X_train, y_train, X_test, y_test, selected_features)
        
        F_s = np.sum(selected_features)  # Number of selected features
        F_all = len(selected_features)   # Total number of features
        
        # Equation 14: Linear weighted fitness function
        fitness = self.alpha * (1 - acc) + (1 - self.alpha) * (F_s / F_all)
        
        return fitness, acc
    
    def _initialize_population(self, n_features):
        """
        Initialize population in binary space
        
        From Section 5.2, Equation 18:
        x_j^i(0) = {1, if rand > 0.5; 0, otherwise}
        
        where rand ∈ U(0,1) is a random number in uniform distribution
        """
        population = np.zeros((self.n_snakes, n_features))
        
        for i in range(self.n_snakes):
            for j in range(n_features):
                # Equation 18: Binary initialization
                population[i, j] = 1 if random.random() > 0.5 else 0
                
        return population
    
    def _hamming_distance(self, snake1, snake2):
        """
        Calculate Hamming distance between two binary vectors
        
        From Section 5.4.1, Equations 20-21:
        D_h(X_i, X_j) = Σ(k=1 to dim) d_h(x_k^i, x_k^j)
        
        where d_h(x_k^i, x_k^j) = {1, if x_k^i ≠ x_k^j; 0, otherwise}
        """
        return np.sum(snake1 != snake2)  # Equations 20-21: Hamming distance calculation
    
    def _sigmoid_transfer(self, x, Q):
        """
        S-shaped transfer function for distance mapping
        
        From Section 5.4.2, Equation 23:
        S(x) = 1 / (e^(-x/Q) + 1)
        
        where Q is the food quantity that changes over iterations
        """
        return 1 / (np.exp(-x / Q) + 1)  # Equation 23: Distance transfer function
    
    def _gaussian_mutation_transfer(self, value):
        """
        Gaussian mutation transfer function
        
        From Section 5.5, Equation 29:
        x_j^i = {1, if Gauss < x_j^i; 0, otherwise}
        
        where Gauss ~ N(0.5, 0.25²) is a random number from Gaussian distribution
        with mean=0.5 and standard deviation=0.25
        """
        gauss = np.random.normal(0.5, 0.25)  # Gaussian distribution N(0.5, 0.25²)
        return 1 if gauss < value else 0     # Equation 29: Mutation transfer function
    
    def _update_temperature_and_food(self, t):
        """
        Update temperature and food quantity parameters
        
        From Section 3, Equations 4-5:
        Temp = exp(-t/T)
        Q = c1 * exp((t-T)/T)
        
        where t is current iteration, T is maximum iterations
        """
        # Equation 4: Temperature calculation
        temp = np.exp(-t / self.max_iterations)
        
        # Equation 5: Food quantity calculation  
        Q = self.c1 * np.exp((t - self.max_iterations) / self.max_iterations)
        
        return temp, Q
    
    def _exploration_phase(self, population, fitness_values):
        """
        Exploration phase when Q < threshold_food
        
        From Section 5.3, Equation 19:
        x_j^i(t+1) = x_j^rand(t) ⊕ c2 * A * rand
        
        where A = exp(f_rand / f_i) from Equation 7
        """
        n_snakes, n_features = population.shape
        new_population = population.copy()
        
        for i in range(n_snakes):
            # Select random individual from same group (male/female)
            group_size = n_snakes // 2
            if i < group_size:  # Male group
                rand_idx = random.randint(0, group_size - 1)
            else:  # Female group  
                rand_idx = random.randint(group_size, n_snakes - 1)
            
            # Equation 7: Calculate A factor
            if fitness_values[i] != 0:
                A = np.exp(fitness_values[rand_idx] / fitness_values[i])
            else:
                A = 1
            
            # Equation 19: Update position dimension by dimension
            for j in range(n_features):
                update_prob = self.c2 * A * random.random()
                new_population[i, j] = population[rand_idx, j] if random.random() < update_prob else population[i, j]
        
        return new_population
    
    def _exploitation_phase_high_temp(self, population, best_position, temp, Q):
        """
        Exploitation phase when Temp > threshold_temp
        
        From Section 5.4.3, Equations 24-25:
        x_j^i(t+1) = {x_j^i(t), if d_h(x_j^food, x_j^i(t)) = 0; x_j^food ⊕ H, otherwise}
        
        where H = c3 * Temp * rand * S(D_h(X_food, X_i))
        """
        n_snakes, n_features = population.shape
        new_population = population.copy()
        
        for i in range(n_snakes):
            for j in range(n_features):
                # Check if dimension values are the same
                if population[i, j] == best_position[j]:
                    # Equation 24: Keep same dimension unchanged
                    new_population[i, j] = population[i, j]
                else:
                    # Calculate hamming distance for entire vectors
                    hamming_dist = self._hamming_distance(best_position, population[i])
                    
                    # Equation 25: Calculate H factor
                    S_value = self._sigmoid_transfer(hamming_dist, Q)
                    H = self.c3 * temp * random.random() * S_value
                    
                    # Equation 24: Update with food position and H
                    new_population[i, j] = best_position[j] if random.random() < H else population[i, j]
        
        return new_population
    
    def _exploitation_phase_low_temp(self, population, fitness_values, temp, Q):
        """
        Exploitation phase when Temp < threshold_temp (Fight and Mating modes)
        
        From Section 3, Equations 9-12:
        Fight mode (Equation 9): X_i(t+1) = X_i(t) + c3 * F * rand * Q * (X_best - X_i(t))
        Mating mode (Equation 11): X_i(t+1) = X_i(t) + c3 * M * rand * Q * (X_i,another(t) - X_i(t))
        
        where F = exp(f_best / f_i) and M = exp(f_i,another / f_i)
        """
        n_snakes, n_features = population.shape
        new_population = population.copy()
        
        # Split into male and female groups (Equations 2-3)
        n_males = n_snakes // 2
        n_females = n_snakes - n_males
        
        males = population[:n_males]
        females = population[n_males:]
        male_fitness = fitness_values[:n_males]
        female_fitness = fitness_values[n_males:]
        
        # Find best in each group
        best_male_idx = np.argmin(male_fitness)
        best_female_idx = np.argmin(female_fitness)
        best_male = males[best_male_idx]
        best_female = females[best_female_idx]
        
        # Update males
        for i in range(n_males):
            if random.random() < 0.5:  # Fight mode
                # Equation 10: F = exp(f_best / f_i)
                if male_fitness[i] != 0:
                    F = np.exp(female_fitness[best_female_idx] / male_fitness[i])
                else:
                    F = 1
                
                # Equation 9: Fight mode update
                for j in range(n_features):
                    update_prob = self.c3 * F * random.random() * Q
                    if random.random() < update_prob:
                        new_population[i, j] = best_female[j]
            else:  # Mating mode
                partner_idx = random.randint(n_males, n_snakes - 1)  # Female partner
                
                # Equation 12: M = exp(f_i,another / f_i)  
                if male_fitness[i] != 0:
                    M = np.exp(fitness_values[partner_idx] / male_fitness[i])
                else:
                    M = 1
                
                # Equation 11: Mating mode update
                for j in range(n_features):
                    update_prob = self.c3 * M * random.random() * Q
                    if random.random() < update_prob:
                        new_population[i, j] = population[partner_idx, j]
        
        # Update females (similar process)
        for i in range(n_females):
            actual_idx = i + n_males
            if random.random() < 0.5:  # Fight mode
                if female_fitness[i] != 0:
                    F = np.exp(male_fitness[best_male_idx] / female_fitness[i])
                else:
                    F = 1
                
                for j in range(n_features):
                    update_prob = self.c3 * F * random.random() * Q
                    if random.random() < update_prob:
                        new_population[actual_idx, j] = best_male[j]
            else:  # Mating mode
                partner_idx = random.randint(0, n_males - 1)  # Male partner
                
                if female_fitness[i] != 0:
                    M = np.exp(male_fitness[partner_idx] / female_fitness[i])
                else:
                    M = 1
                
                for j in range(n_features):
                    update_prob = self.c3 * M * random.random() * Q
                    if random.random() < update_prob:
                        new_population[actual_idx, j] = population[partner_idx, j]
        
        return new_population
    
    def _apply_gaussian_mutation_transfer(self, population):
        """
        Apply Gaussian mutation transfer function to convert continuous to binary
        
        From Section 5.5: "we propose a mutation transfer function with Gaussian distribution"
        This enhances "local random search capability and increase the population diversity"
        """
        n_snakes, n_features = population.shape
        binary_population = np.zeros_like(population)
        
        for i in range(n_snakes):
            for j in range(n_features):
                # Apply Gaussian mutation transfer function (Equation 29)
                binary_population[i, j] = self._gaussian_mutation_transfer(population[i, j])
        
        return binary_population
    
    def _replace_worst_with_newborn(self, population, fitness_values):
        """
        Replace worst individual with newborn (after mating)
        
        From Section 3, Equation 13:
        X_worst = X_min + rand * (X_max - X_min)
        
        In binary space, this becomes random binary initialization
        """
        worst_idx = np.argmax(fitness_values)
        n_features = population.shape[1]
        
        # Equation 13 adapted for binary space (Section 5.2, Equation 18)
        for j in range(n_features):
            population[worst_idx, j] = 1 if random.random() > 0.5 else 0
        
        return population
    
    def optimize(self, X, y,eval_x, eval_y, test_size=0.2, random_state=42):
        """
        Main optimization function for feature selection
        
        From Algorithm 1 and Figure 2: The complete IBSO flowchart
        
        Parameters:
        - X: Feature matrix
        - y: Target vector  
        - test_size: Test set ratio (Section 6.3: "80% for training, 20% for testing")
        - random_state: Random seed for reproducibility
        
        Returns:
        - best_features: Binary array indicating selected features
        - best_fitness: Best fitness value achieved
        - best_accuracy: Best accuracy achieved
        - convergence_curve: Fitness values over iterations
        """

        X_train, X_test, y_train, y_test = X, eval_x, y, eval_y

        # if (eval_x is not None) and (eval_y is not None):
        #     # Use provided evaluation data if available
        #     X_train, X_test, y_train, y_test = X, eval_x, y, eval_y

        # else:
        #     # Split data (Section 6.3: 80% training, 20% testing)
        #     X_train, X_test, y_train, y_test = train_test_split(
        #         X, y, test_size=test_size, random_state=random_state
        #     )
        

        
        n_features = X.shape[1]
        
        # Step 1: Initialize population (Section 5.2, Equation 18)
        population = self._initialize_population(n_features)
        
        # Evaluate initial population
        fitness_values = np.zeros(self.n_snakes)
        accuracy_values = np.zeros(self.n_snakes)
        
        for i in range(self.n_snakes):
            fitness_values[i], accuracy_values[i] = self._fitness_function(
                X_train, y_train, X_test, y_test, population[i]
            )
        
        # Track best solution
        best_idx = np.argmin(fitness_values)
        best_features = population[best_idx].copy()
        best_fitness = fitness_values[best_idx]
        best_accuracy = accuracy_values[best_idx]
        
        convergence_curve = [best_fitness]
        
        # Main optimization loop (Algorithm 1)
        for t in range(self.max_iterations):
            # Step 2: Update temperature and food quantity (Equations 4-5)
            temp, Q = self._update_temperature_and_food(t)
            
            # Step 3: Choose optimization phase based on food quantity
            if Q < self.threshold_food:
                # Exploration phase (Section 5.3)
                population = self._exploration_phase(population, fitness_values)
            else:
                # Exploitation phases
                if temp > self.threshold_temp:
                    # High temperature exploitation (Section 5.4)
                    population = self._exploitation_phase_high_temp(
                        population, best_features, temp, Q
                    )
                else:
                    # Low temperature exploitation - Fight/Mating (Section 3)
                    population = self._exploitation_phase_low_temp(
                        population, fitness_values, temp, Q
                    )
                    
                    # Replace worst individual with newborn (Equation 13)
                    population = self._replace_worst_with_newborn(population, fitness_values)
            
            # Step 4: Apply Gaussian mutation transfer function (Section 5.5)
            population = self._apply_gaussian_mutation_transfer(population)
            
            # Step 5: Evaluate new population
            for i in range(self.n_snakes):
                fitness_values[i], accuracy_values[i] = self._fitness_function(
                    X_train, y_train, X_test, y_test, population[i]
                )
            
            # Step 6: Update best solution
            current_best_idx = np.argmin(fitness_values)
            if fitness_values[current_best_idx] < best_fitness:
                best_fitness = fitness_values[current_best_idx]
                best_features = population[current_best_idx].copy()
                best_accuracy = accuracy_values[current_best_idx]
            
            convergence_curve.append(best_fitness)
            
            # Optional: Print progress
            if (t + 1) % 20 == 0:
                selected_count = np.sum(best_features)
                print(f"Iteration {t+1}: Best Fitness = {best_fitness:.4f}, "
                      f"Accuracy = {best_accuracy:.4f}, Features = {selected_count}/{n_features}")
        
        return best_features, best_fitness, best_accuracy, convergence_curve



In [20]:
# Example usage function
def example_usage():
    """
    Example of how to use the Binary Snake Optimizer for feature selection
    """
    # from sklearn.datasets import load_breast_cancer
    
    # # Load example dataset
    # data = load_breast_cancer()
    # X, y = data.data, data.target

    original_data_dir = './outputs/' +'checkpoints/original_text_data_features/breastEW_'
    X = np.load(original_data_dir+"train_data.npy")
    y = np.load(original_data_dir+"train_label.npy")
    ty = np.load(original_data_dir+"eval_label.npy")
    tX = np.load(original_data_dir+"eval_data.npy")
    
    print(f"Training dataset shape: {X.shape}")
    print(f"Training labels shape: {y.shape}")
    print(f"Testing dataset shape: {tX.shape}")
    print(f"Testing labels shape: {ty.shape}")
    
    print(f"Dataset: {X.shape[0]} samples, {X.shape[1]} features")
    
    # Initialize BSO with parameters from paper
    bso = BinarySnakeOptimizer(
        n_snakes=30,        # Population size (Section 6.3)
        max_iterations=100, # Maximum iterations (Section 6.3) 
        alpha=0.99         # Weight parameter (Section 6.3)
    )
    
    # Optimize feature selection
    print("Starting optimization...")
    best_features, best_fitness, best_accuracy, convergence = bso.optimize(X, y, tX, ty,)
    
    # Results
    selected_features = np.where(best_features == 1)[0]
    print(f"\nOptimization completed!")
    print(f"Best fitness: {best_fitness:.4f}")
    print(f"Best accuracy: {best_accuracy:.4f}")
    print(f"Selected {len(selected_features)} out of {len(best_features)} features")
    print(f"Feature reduction: {(1 - len(selected_features)/len(best_features))*100:.1f}%")
    
    return best_features, convergence

example_usage()

Training dataset shape: (425, 30)
Training labels shape: (425, 2)
Testing dataset shape: (56, 30)
Testing labels shape: (56, 2)
Dataset: 425 samples, 30 features
Starting optimization...
Iteration 20: Best Fitness = 0.0564, Accuracy = 0.9464, Features = 10.0/30
Iteration 40: Best Fitness = 0.0547, Accuracy = 0.9464, Features = 5.0/30
Iteration 60: Best Fitness = 0.0547, Accuracy = 0.9464, Features = 5.0/30
Iteration 80: Best Fitness = 0.0547, Accuracy = 0.9464, Features = 5.0/30
Iteration 100: Best Fitness = 0.0547, Accuracy = 0.9464, Features = 5.0/30

Optimization completed!
Best fitness: 0.0547
Best accuracy: 0.9464
Selected 5 out of 30 features
Feature reduction: 83.3%


(array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 [0.07471428571428569,
  0.07471428571428569,
  0.07471428571428569,
  0.07471428571428569,
  0.07471428571428569,
  0.07471428571428569,
  0.060035714285714324,
  0.05870238095238099,
  0.05870238095238099,
  0.05870238095238099,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05636904761904765,
  0.05603571428571432,
  0.05536904761904765,
  0.05503571428571432,
  0.05503571428571432,
  0.05503571428571432,
  0.054702

In [16]:
import numpy as np
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

class BinarySnakeOptimizer2:
    """
    Binary Snake Optimizer (BSO) for Feature Selection
    
    Implementation based on: "An improved binary snake optimizer with Gaussian mutation 
    transfer function and hamming distance for feature selection" by Bao et al. (2024)
    """
    
    def __init__(self, n_snakes=30, max_iterations=100, alpha=0.99, accuracy_func=None):
        """
        Initialize Binary Snake Optimizer
        
        Parameters from Section 6.2 (Table 3):
        - n_snakes: Population size (set to 30 as per experimental setup)
        - max_iterations: Maximum iterations (set to 100 as per experimental setup)
        - alpha: Weight parameter in fitness function (Equation 14, set to 0.99)
        - accuracy_func: Custom accuracy evaluation function
        
        Constants from Section 3 (Snake Optimization Algorithm):
        - c1: Constant equal to 0.5 (Equation 5)
        - c2: Constant equal to 0.05 (Equation 6) 
        - c3: Constant equal to 2 (Equation 8)
        - threshold_temp: Temperature threshold = 0.6 (Section 6.2)
        - threshold_food: Food threshold = 0.25 (Section 6.2)
        """
        self.n_snakes = n_snakes
        self.max_iterations = max_iterations
        self.alpha = alpha  # Weight parameter from Equation 14
        self.accuracy_func = accuracy_func or self._default_accuracy_func
        
        # Algorithm constants from Section 3 and Table 3
        self.c1 = 0.5      # Equation 5: Q = c1 * exp((t-T)/T)
        self.c2 = 0.05     # Equation 6: exploration phase constant
        self.c3 = 2        # Equation 8: exploitation phase constant
        self.threshold_temp = 0.6   # Temperature threshold (Section 6.2)
        self.threshold_food = 0.25  # Food threshold (Section 6.2)
        
    def _default_accuracy_func(self, X_train, y_train, X_test, y_test, selected_features):
        """
        Default accuracy function using KNN classifier
        
        From Section 4: "choosing classifiers with lower computational cost, 
        such as K-nearest neighbors (KNN), can help mitigate this problem"
        
        From Section 6.3: "we utilize the KNN (k=5) as the classifier"
        """
        if np.sum(selected_features) == 0:
            return 0.0

        selected_features = np.where(selected_features)[0]
            
        X_train_selected = X_train[:, selected_features]
        X_test_selected = X_test[:, selected_features]
        
        knn = KNeighborsClassifier(n_neighbors=5)  # k=5 as per Section 6.3
        knn.fit(X_train_selected, y_train)
        y_pred = knn.predict(X_test_selected)
        
        return accuracy_score(y_test, y_pred)
    
    def _fitness_function(self, X_train, y_train, X_test, y_test, selected_features):
        """
        Fitness function based on linear weighted method
        
        From Section 4, Equation 14:
        Fitness = α * (1 - acc) + (1 - α) * (F_s / F_all)
        
        where:
        - acc: prediction accuracy of machine learning classifier
        - F_s: number of selected features
        - F_all: total number of features
        - α: weight parameter (set to 0.99 in Section 6.3)
        """
        acc = self.accuracy_func(X_train, y_train, X_test, y_test, selected_features)
        
        F_s = np.sum(selected_features)  # Number of selected features
        F_all = len(selected_features)   # Total number of features
        
        # Equation 14: Linear weighted fitness function
        fitness = self.alpha * (1 - acc) + (1 - self.alpha) * (F_s / F_all)

        cols=np.flatnonzero(selected_features)
        #val=1
        #if np.shape(cols)[0]==0:
        #   return val, 1-val
        clf=KNeighborsClassifier(n_neighbors=5)
        train_data=X_train[:,cols]
        test_data=X_test[:,cols]
        clf.fit(train_data,y_train)
        val=1-clf.score(test_data,y_test)
        
        set_cnt=sum(selected_features)
        set_cnt=set_cnt/np.shape(selected_features)[0]
        omega =  0.99
        val=omega*val+(1-omega)*set_cnt
        return val, 1-val
        
        # return fitness, acc
    
    def _initialize_population(self, n_features):
        """
        Initialize population in binary space
        
        From Section 5.2, Equation 18:
        x_j^i(0) = {1, if rand > 0.5; 0, otherwise}
        
        where rand ∈ U(0,1) is a random number in uniform distribution
        """
        population = np.zeros((self.n_snakes, n_features))
        
        for i in range(self.n_snakes):
            for j in range(n_features):
                # Equation 18: Binary initialization
                population[i, j] = 1 if random.random() > 0.5 else 0
                
        return population
    
    def _hamming_distance(self, snake1, snake2):
        """
        Calculate Hamming distance between two binary vectors
        
        From Section 5.4.1, Equations 20-21:
        D_h(X_i, X_j) = Σ(k=1 to dim) d_h(x_k^i, x_k^j)
        
        where d_h(x_k^i, x_k^j) = {1, if x_k^i ≠ x_k^j; 0, otherwise}
        """
        return np.sum(snake1 != snake2)  # Equations 20-21: Hamming distance calculation
    
    def _sigmoid_transfer(self, x, Q):
        """
        S-shaped transfer function for distance mapping
        
        From Section 5.4.2, Equation 23:
        S(x) = 1 / (e^(-x/Q) + 1)
        
        where Q is the food quantity that changes over iterations
        """
        return 1 / (np.exp(-x / Q) + 1)  # Equation 23: Distance transfer function
    
    def _gaussian_mutation_transfer(self, value):
        """
        Gaussian mutation transfer function
        
        From Section 5.5, Equation 29:
        x_j^i = {1, if Gauss < x_j^i; 0, otherwise}
        
        where Gauss ~ N(0.5, 0.25²) is a random number from Gaussian distribution
        with mean=0.5 and standard deviation=0.25
        """
        gauss = np.random.normal(0.5, 0.25)  # Gaussian distribution N(0.5, 0.25²)
        return 1 if gauss < value else 0     # Equation 29: Mutation transfer function
    
    def _update_temperature_and_food(self, t):
        """
        Update temperature and food quantity parameters
        
        From Section 3, Equations 4-5:
        Temp = exp(-t/T)
        Q = c1 * exp((t-T)/T)
        
        where t is current iteration, T is maximum iterations
        """
        # Equation 4: Temperature calculation
        temp = np.exp(-t / self.max_iterations)
        
        # Equation 5: Food quantity calculation  
        Q = self.c1 * np.exp((t - self.max_iterations) / self.max_iterations)
        
        return temp, Q
    
    def _exploration_phase(self, population, fitness_values):
        """
        Exploration phase when Q < threshold_food
        
        From Section 5.3, Equation 19:
        x_j^i(t+1) = x_j^rand(t) ⊕ c2 * A * rand
        
        where A = exp(f_rand / f_i) from Equation 7
        """
        n_snakes, n_features = population.shape
        new_population = population.copy()
        
        for i in range(n_snakes):
            # Select random individual from same group (male/female)
            group_size = n_snakes // 2
            if i < group_size:  # Male group
                rand_idx = random.randint(0, group_size - 1)
            else:  # Female group  
                rand_idx = random.randint(group_size, n_snakes - 1)
            
            # Equation 7: Calculate A factor
            if fitness_values[i] != 0:
                A = np.exp(fitness_values[rand_idx] / fitness_values[i])
            else:
                A = 1
            
            # Equation 19: Update position dimension by dimension
            for j in range(n_features):
                update_prob = self.c2 * A * random.random()
                new_population[i, j] = population[rand_idx, j] if random.random() < update_prob else population[i, j]
        
        return new_population
    
    def _exploitation_phase_high_temp(self, population, best_position, temp, Q):
        """
        Exploitation phase when Temp > threshold_temp
        
        From Section 5.4.3, Equations 24-25:
        x_j^i(t+1) = {x_j^i(t), if d_h(x_j^food, x_j^i(t)) = 0; x_j^food ⊕ H, otherwise}
        
        where H = c3 * Temp * rand * S(D_h(X_food, X_i))
        """
        n_snakes, n_features = population.shape
        new_population = population.copy()
        
        for i in range(n_snakes):
            for j in range(n_features):
                # Check if dimension values are the same
                if population[i, j] == best_position[j]:
                    # Equation 24: Keep same dimension unchanged
                    new_population[i, j] = population[i, j]
                else:
                    # Calculate hamming distance for entire vectors
                    hamming_dist = self._hamming_distance(best_position, population[i])
                    
                    # Equation 25: Calculate H factor
                    S_value = self._sigmoid_transfer(hamming_dist, Q)
                    H = self.c3 * temp * random.random() * S_value
                    
                    # Equation 24: Update with food position and H
                    new_population[i, j] = best_position[j] if random.random() < H else population[i, j]
        
        return new_population
    
    def _exploitation_phase_low_temp(self, population, fitness_values, temp, Q):
        """
        Exploitation phase when Temp < threshold_temp (Fight and Mating modes)
        
        From Section 3, Equations 9-12:
        Fight mode (Equation 9): X_i(t+1) = X_i(t) + c3 * F * rand * Q * (X_best - X_i(t))
        Mating mode (Equation 11): X_i(t+1) = X_i(t) + c3 * M * rand * Q * (X_i,another(t) - X_i(t))
        
        where F = exp(f_best / f_i) and M = exp(f_i,another / f_i)
        """
        n_snakes, n_features = population.shape
        new_population = population.copy()
        
        # Split into male and female groups (Equations 2-3)
        n_males = n_snakes // 2
        n_females = n_snakes - n_males
        
        males = population[:n_males]
        females = population[n_males:]
        male_fitness = fitness_values[:n_males]
        female_fitness = fitness_values[n_males:]
        
        # Find best in each group
        best_male_idx = np.argmin(male_fitness)
        best_female_idx = np.argmin(female_fitness)
        best_male = males[best_male_idx]
        best_female = females[best_female_idx]
        
        # Update males
        for i in range(n_males):
            if random.random() < 0.5:  # Fight mode
                # Equation 10: F = exp(f_best / f_i)
                if male_fitness[i] != 0:
                    F = np.exp(female_fitness[best_female_idx] / male_fitness[i])
                else:
                    F = 1
                
                # Equation 9: Fight mode update
                for j in range(n_features):
                    update_prob = self.c3 * F * random.random() * Q
                    if random.random() < update_prob:
                        new_population[i, j] = best_female[j]
            else:  # Mating mode
                partner_idx = random.randint(n_males, n_snakes - 1)  # Female partner
                
                # Equation 12: M = exp(f_i,another / f_i)  
                if male_fitness[i] != 0:
                    M = np.exp(fitness_values[partner_idx] / male_fitness[i])
                else:
                    M = 1
                
                # Equation 11: Mating mode update
                for j in range(n_features):
                    update_prob = self.c3 * M * random.random() * Q
                    if random.random() < update_prob:
                        new_population[i, j] = population[partner_idx, j]
        
        # Update females (similar process)
        for i in range(n_females):
            actual_idx = i + n_males
            if random.random() < 0.5:  # Fight mode
                if female_fitness[i] != 0:
                    F = np.exp(male_fitness[best_male_idx] / female_fitness[i])
                else:
                    F = 1
                
                for j in range(n_features):
                    update_prob = self.c3 * F * random.random() * Q
                    if random.random() < update_prob:
                        new_population[actual_idx, j] = best_male[j]
            else:  # Mating mode
                partner_idx = random.randint(0, n_males - 1)  # Male partner
                
                if female_fitness[i] != 0:
                    M = np.exp(male_fitness[partner_idx] / female_fitness[i])
                else:
                    M = 1
                
                for j in range(n_features):
                    update_prob = self.c3 * M * random.random() * Q
                    if random.random() < update_prob:
                        new_population[actual_idx, j] = population[partner_idx, j]
        
        return new_population
    
    def _apply_gaussian_mutation_transfer(self, population):
        """
        Apply Gaussian mutation transfer function to convert continuous to binary
        
        From Section 5.5: "we propose a mutation transfer function with Gaussian distribution"
        This enhances "local random search capability and increase the population diversity"
        """
        n_snakes, n_features = population.shape
        binary_population = np.zeros_like(population)
        
        for i in range(n_snakes):
            for j in range(n_features):
                # Apply Gaussian mutation transfer function (Equation 29)
                binary_population[i, j] = self._gaussian_mutation_transfer(population[i, j])
        
        return binary_population
    
    def _replace_worst_with_newborn(self, population, fitness_values):
        """
        Replace worst individual with newborn (after mating)
        
        From Section 3, Equation 13:
        X_worst = X_min + rand * (X_max - X_min)
        
        In binary space, this becomes random binary initialization
        """
        worst_idx = np.argmax(fitness_values)
        n_features = population.shape[1]
        
        # Equation 13 adapted for binary space (Section 5.2, Equation 18)
        for j in range(n_features):
            population[worst_idx, j] = 1 if random.random() > 0.5 else 0
        
        return population
    
    def optimize(self, X, y,eval_x, eval_y, test_size=0.2, random_state=42):
        """
        Main optimization function for feature selection
        
        From Algorithm 1 and Figure 2: The complete IBSO flowchart
        
        Parameters:
        - X: Feature matrix
        - y: Target vector  
        - test_size: Test set ratio (Section 6.3: "80% for training, 20% for testing")
        - random_state: Random seed for reproducibility
        
        Returns:
        - best_features: Binary array indicating selected features
        - best_fitness: Best fitness value achieved
        - best_accuracy: Best accuracy achieved
        - convergence_curve: Fitness values over iterations
        """

        X_train, X_test, y_train, y_test = X, eval_x, y, eval_y

        # if (eval_x is not None) and (eval_y is not None):
        #     # Use provided evaluation data if available
        #     X_train, X_test, y_train, y_test = X, eval_x, y, eval_y

        # else:
        #     # Split data (Section 6.3: 80% training, 20% testing)
        #     X_train, X_test, y_train, y_test = train_test_split(
        #         X, y, test_size=test_size, random_state=random_state
        #     )
        

        
        n_features = X.shape[1]
        
        # Step 1: Initialize population (Section 5.2, Equation 18)
        population = self._initialize_population(n_features)
        
        # Evaluate initial population
        fitness_values = np.zeros(self.n_snakes)
        accuracy_values = np.zeros(self.n_snakes)
        
        for i in range(self.n_snakes):
            fitness_values[i], accuracy_values[i] = self._fitness_function(
                X_train, y_train, X_test, y_test, population[i]
            )
        
        # Track best solution
        best_idx = np.argmin(fitness_values)
        best_features = population[best_idx].copy()
        best_fitness = fitness_values[best_idx]
        best_accuracy = accuracy_values[best_idx]
        
        convergence_curve = [best_fitness]
        
        # Main optimization loop (Algorithm 1)
        for t in range(self.max_iterations):
            # Step 2: Update temperature and food quantity (Equations 4-5)
            temp, Q = self._update_temperature_and_food(t)
            
            # Step 3: Choose optimization phase based on food quantity
            if Q < self.threshold_food:
                # Exploration phase (Section 5.3)
                population = self._exploration_phase(population, fitness_values)
            else:
                # Exploitation phases
                if temp > self.threshold_temp:
                    # High temperature exploitation (Section 5.4)
                    population = self._exploitation_phase_high_temp(
                        population, best_features, temp, Q
                    )
                else:
                    # Low temperature exploitation - Fight/Mating (Section 3)
                    population = self._exploitation_phase_low_temp(
                        population, fitness_values, temp, Q
                    )
                    
                    # Replace worst individual with newborn (Equation 13)
                    population = self._replace_worst_with_newborn(population, fitness_values)
            
            # Step 4: Apply Gaussian mutation transfer function (Section 5.5)
            population = self._apply_gaussian_mutation_transfer(population)
            
            # Step 5: Evaluate new population
            for i in range(self.n_snakes):
                fitness_values[i], accuracy_values[i] = self._fitness_function(
                    X_train, y_train, X_test, y_test, population[i]
                )
            
            # Step 6: Update best solution
            current_best_idx = np.argmin(fitness_values)
            if fitness_values[current_best_idx] < best_fitness:
                best_fitness = fitness_values[current_best_idx]
                best_features = population[current_best_idx].copy()
                best_accuracy = accuracy_values[current_best_idx]
            
            convergence_curve.append(best_fitness)
            
            # Optional: Print progress
            if (t + 1) % 20 == 0:
                selected_count = np.sum(best_features)
                print(f"Iteration {t+1}: Best Fitness = {best_fitness:.4f}, "
                      f"Accuracy = {best_accuracy:.4f}, Features = {selected_count}/{n_features}")
        
        return best_features, best_fitness, best_accuracy, convergence_curve



In [19]:
# Example usage function
def example_usage_2():
    """
    Example of how to use the Binary Snake Optimizer for feature selection
    """
    # from sklearn.datasets import load_breast_cancer
    
    # # Load example dataset
    # data = load_breast_cancer()
    # X, y = data.data, data.target

    original_data_dir = './outputs/' +'checkpoints/original_text_data_features/breastEW_'
    X = np.load(original_data_dir+"train_data.npy")
    y = np.load(original_data_dir+"train_label.npy")
    ty = np.load(original_data_dir+"eval_label.npy")
    tX = np.load(original_data_dir+"eval_data.npy")
    
    print(f"Training dataset shape: {X.shape}")
    print(f"Training labels shape: {y.shape}")
    print(f"Testing dataset shape: {tX.shape}")
    print(f"Testing labels shape: {ty.shape}")
    
    print(f"Dataset: {X.shape[0]} samples, {X.shape[1]} features")
    
    # Initialize BSO with parameters from paper
    bso2 = BinarySnakeOptimizer2(
        n_snakes=30,        # Population size (Section 6.3)
        max_iterations=100, # Maximum iterations (Section 6.3) 
        alpha=0.99         # Weight parameter (Section 6.3)
    )
    
    # Optimize feature selection
    print("Starting optimization...")
    best_features, best_fitness, best_accuracy, convergence = bso2.optimize(X, y, tX, ty,)
    
    # Results
    selected_features = np.where(best_features == 1)[0]
    print(f"\nOptimization completed!")
    print(f"Best fitness: {best_fitness:.4f}")
    print(f"Best accuracy: {best_accuracy:.4f}")
    print(f"Selected {len(selected_features)} out of {len(best_features)} features")
    print(f"Feature reduction: {(1 - len(selected_features)/len(best_features))*100:.1f}%")
    
    return best_features, convergence

example_usage_2()

Training dataset shape: (425, 30)
Training labels shape: (425, 2)
Testing dataset shape: (56, 30)
Testing labels shape: (56, 2)
Dataset: 425 samples, 30 features
Starting optimization...


Iteration 20: Best Fitness = 0.0754, Accuracy = 0.9246, Features = 14.0/30
Iteration 40: Best Fitness = 0.0557, Accuracy = 0.9443, Features = 8.0/30
Iteration 60: Best Fitness = 0.0547, Accuracy = 0.9453, Features = 5.0/30
Iteration 80: Best Fitness = 0.0547, Accuracy = 0.9453, Features = 5.0/30
Iteration 100: Best Fitness = 0.0547, Accuracy = 0.9453, Features = 5.0/30

Optimization completed!
Best fitness: 0.0547
Best accuracy: 0.9453
Selected 5 out of 30 features
Feature reduction: 83.3%


(array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 [0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07571428571428569,
  0.07538095238095235,
  0.07538095238095235,
  0.07538095238095235,
  0.07538095238095235,
  0.07538095238095235,
  0.07538095238095235,
  0.07538095238095235,
  0.07538095238095235,
  0.07504761904761903,
  0.07504761904761903,
  0.07504761904761903,
  0.07504761904761903,
  0.07504761904761903,
  0.07504761904761903,
  0.07504761904761903,
  0.07471428571428569,
  0.05736904761904765,
  0.05703571428571432,
  0.05703571428571432,
  0.05670238095238099,
  0.0563690

In [ ]:
import numpy as np
import random
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Constants matching your EOSA structure
EOSA_INDIVIDUAL = 1
EOSA_ID_POS = 0
EOSA_ID_FIT = 1

def EOSA_fitness(position, trainX, testX, trainy, testy):
    """
    Fitness function compatible with your EOSA structure
    Using the same linear weighted method as original BSO
    """
    # Convert continuous position to binary
    binary_pos = np.array([1 if x > 0.5 else 0 for x in position])
    
    if np.sum(binary_pos) == 0:
        return 1.0, 1.0  # Worst fitness if no features selected
    
    selected_features = np.where(binary_pos)[0]
    
    X_train_selected = trainX[:, selected_features]
    X_test_selected = testX[:, selected_features]
    
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train_selected, trainy)
    y_pred = knn.predict(X_test_selected)
    
    acc = accuracy_score(testy, y_pred)
    
    # Linear weighted fitness function (Equation 14 from paper)
    alpha = 0.99
    F_s = np.sum(binary_pos)  # Number of selected features
    F_all = len(binary_pos)   # Total number of features
    
    fitness = alpha * (1 - acc) + (1 - alpha) * (F_s / F_all)
    cost = fitness  # Same as fitness for compatibility
    
    return fitness, cost

def BSO(agents, fitAgent, trainX, testX, trainy, testy, dim, isNormalPop, curIter, MaxIter):
    """
    Binary Snake Optimizer adapted to match your PSO interface
    
    Parameters:
    - agents: Population of agents in your EOSA format
    - fitAgent: Not used, kept for interface compatibility
    - trainX, testX, trainy, testy: Training and testing data
    - dim: Number of features (dimensions)
    - isNormalPop: Should be False for EOSA format
    - curIter: Current iteration
    - MaxIter: Maximum iterations
    
    Returns:
    - Updated agents list
    """
    
    # BSO Algorithm constants
    c1 = 0.5      # Equation 5: Q = c1 * exp((t-T)/T)
    c2 = 0.05     # Equation 6: exploration phase constant
    c3 = 2        # Equation 8: exploitation phase constant
    threshold_temp = 0.6   # Temperature threshold
    threshold_food = 0.25  # Food threshold
    
    popSize = len(agents)
    
    # Extract positions and fitness values
    population = np.zeros((popSize, dim))
    fitness_values = np.zeros(popSize)
    
    for i in range(popSize):
        if isNormalPop:
            population[i] = agents[i]
            # Calculate fitness if needed
            fitness_values[i], _ = EOSA_fitness(agents[i], trainX, testX, trainy, testy)
        else:
            population[i] = agents[i][EOSA_INDIVIDUAL][EOSA_ID_POS]
            fitness_values[i] = agents[i][EOSA_INDIVIDUAL][EOSA_ID_FIT]
    
    # Convert to binary space
    binary_population = np.zeros_like(population)
    for i in range(popSize):
        for j in range(dim):
            binary_population[i, j] = 1 if population[i, j] > 0.5 else 0
    
    # Find best solution
    best_idx = np.argmin(fitness_values)
    best_position = binary_population[best_idx].copy()
    
    # Update temperature and food quantity (Equations 4-5)
    t = curIter - 1  # Convert to 0-based indexing
    temp = np.exp(-t / MaxIter)
    Q = c1 * np.exp((t - MaxIter) / MaxIter)
    
    # Choose optimization phase based on food quantity
    if Q < threshold_food:
        # Exploration phase (Section 5.3)
        binary_population = _exploration_phase(binary_population, fitness_values, c2)
    else:
        # Exploitation phases
        if temp > threshold_temp:
            # High temperature exploitation (Section 5.4)
            binary_population = _exploitation_phase_high_temp(
                binary_population, best_position, temp, Q, c3
            )
        else:
            # Low temperature exploitation - Fight/Mating (Section 3)
            binary_population = _exploitation_phase_low_temp(
                binary_population, fitness_values, temp, Q, c3
            )
            
            # Replace worst individual with newborn
            worst_idx = np.argmax(fitness_values)
            for j in range(dim):
                binary_population[worst_idx, j] = 1 if random.random() > 0.5 else 0
    
    # Apply Gaussian mutation transfer function
    binary_population = _apply_gaussian_mutation_transfer(binary_population)
    
    # Convert back to continuous space and update agents
    popnew = []
    
    for i in range(popSize):
        # Convert binary to continuous (for compatibility with your system)
        continuous_pos = binary_population[i].astype(float)
        
        if isNormalPop:
            popnew.append(continuous_pos)
        else:
            # Calculate new fitness
            fit, cost = EOSA_fitness(continuous_pos, trainX, testX, trainy, testy)
            
            # Maintain agent structure
            index, _ = agents[i]
            individual = index, [continuous_pos, fit]
            popnew.append(individual)
    
    return popnew

def _hamming_distance(snake1, snake2):
    """Calculate Hamming distance between two binary vectors"""
    return np.sum(snake1 != snake2)

def _sigmoid_transfer(x, Q):
    """S-shaped transfer function for distance mapping"""
    return 1 / (np.exp(-x / Q) + 1)

def _gaussian_mutation_transfer(value):
    """Gaussian mutation transfer function"""
    gauss = np.random.normal(0.5, 0.25)
    return 1 if gauss < value else 0

def _exploration_phase(population, fitness_values, c2):
    """Exploration phase when Q < threshold_food"""
    n_snakes, n_features = population.shape
    new_population = population.copy()
    
    for i in range(n_snakes):
        # Select random individual from same group (male/female)
        group_size = n_snakes // 2
        if i < group_size:  # Male group
            rand_idx = random.randint(0, group_size - 1)
        else:  # Female group  
            rand_idx = random.randint(group_size, n_snakes - 1)
        
        # Calculate A factor (Equation 7)
        if fitness_values[i] != 0:
            A = np.exp(fitness_values[rand_idx] / fitness_values[i])
        else:
            A = 1
        
        # Update position dimension by dimension (Equation 19)
        for j in range(n_features):
            random.seed(time.time() + i * n_features + j)
            update_prob = c2 * A * random.random()
            if random.random() < update_prob:
                new_population[i, j] = population[rand_idx, j]
    
    return new_population

def _exploitation_phase_high_temp(population, best_position, temp, Q, c3):
    """Exploitation phase when Temp > threshold_temp"""
    n_snakes, n_features = population.shape
    new_population = population.copy()
    
    for i in range(n_snakes):
        hamming_dist = _hamming_distance(best_position, population[i])
        S_value = _sigmoid_transfer(hamming_dist, Q)
        
        for j in range(n_features):
            if population[i, j] == best_position[j]:
                # Keep same dimension unchanged (Equation 24)
                new_population[i, j] = population[i, j]
            else:
                # Calculate H factor and update (Equation 25)
                H = c3 * temp * random.random() * S_value
                if random.random() < H:
                    new_population[i, j] = best_position[j]
    
    return new_population

def _exploitation_phase_low_temp(population, fitness_values, temp, Q, c3):
    """Exploitation phase when Temp < threshold_temp (Fight and Mating modes)"""
    n_snakes, n_features = population.shape
    new_population = population.copy()
    
    # Split into male and female groups
    n_males = n_snakes // 2
    n_females = n_snakes - n_males
    
    male_fitness = fitness_values[:n_males]
    female_fitness = fitness_values[n_males:]
    
    # Find best in each group
    best_male_idx = np.argmin(male_fitness)
    best_female_idx = np.argmin(female_fitness)
    best_male = population[best_male_idx]
    best_female = population[n_males + best_female_idx]
    
    # Update males
    for i in range(n_males):
        if random.random() < 0.5:  # Fight mode
            if male_fitness[i] != 0:
                F = np.exp(female_fitness[best_female_idx] / male_fitness[i])
            else:
                F = 1
            
            for j in range(n_features):
                update_prob = c3 * F * random.random() * Q
                if random.random() < update_prob:
                    new_population[i, j] = best_female[j]
        else:  # Mating mode
            partner_idx = random.randint(n_males, n_snakes - 1)
            
            if male_fitness[i] != 0:
                M = np.exp(fitness_values[partner_idx] / male_fitness[i])
            else:
                M = 1
            
            for j in range(n_features):
                update_prob = c3 * M * random.random() * Q
                if random.random() < update_prob:
                    new_population[i, j] = population[partner_idx, j]
    
    # Update females
    for i in range(n_females):
        actual_idx = i + n_males
        if random.random() < 0.5:  # Fight mode
            if female_fitness[i] != 0:
                F = np.exp(male_fitness[best_male_idx] / female_fitness[i])
            else:
                F = 1
            
            for j in range(n_features):
                update_prob = c3 * F * random.random() * Q
                if random.random() < update_prob:
                    new_population[actual_idx, j] = best_male[j]
        else:  # Mating mode
            partner_idx = random.randint(0, n_males - 1)
            
            if female_fitness[i] != 0:
                M = np.exp(male_fitness[partner_idx] / female_fitness[i])
            else:
                M = 1
            
            for j in range(n_features):
                update_prob = c3 * M * random.random() * Q
                if random.random() < update_prob:
                    new_population[actual_idx, j] = population[partner_idx, j]
    
    return new_population

def _apply_gaussian_mutation_transfer(population):
    """Apply Gaussian mutation transfer function to convert continuous to binary"""
    n_snakes, n_features = population.shape
    binary_population = np.zeros_like(population)
    
    for i in range(n_snakes):
        for j in range(n_features):
            binary_population[i, j] = _gaussian_mutation_transfer(population[i, j])
    
    return binary_population

# Tests for snake optimization 

In [ ]:
# Next algorithms  For Binary snake optimization
import numpy as np
import random
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Constants matching your EOSA structure
EOSA_INDIVIDUAL = 1
EOSA_ID_POS = 0
EOSA_ID_FIT = 1

def EOSA_fitness(agent, trainX, testX, trainy, testy):
    cols=np.flatnonzero(agent)
    #val=1
    omega = 0.99
    #if np.shape(cols)[0]==0:
     #   return val, 1-val
    clf=KNeighborsClassifier(n_neighbors=5)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=1-clf.score(test_data,testy)
    
    set_cnt=sum(agent)
    set_cnt=set_cnt/np.shape(agent)[0]
    val=omega*val+(1-omega)*set_cnt
    return val, 1-val

def _hamming_distance(snake1, snake2):
    """Calculate Hamming distance between two binary vectors"""
    return np.sum(snake1 != snake2)

def _sigmoid_transfer(x, Q):
    """S-shaped transfer function for distance mapping"""
    return 1 / (np.exp(-x / Q) + 1)

def _gaussian_mutation_transfer(value):
    """Gaussian mutation transfer function"""
    gauss = np.random.normal(0.5, 0.25)
    return 1 if gauss < value else 0

def _exploration_phase(population, fitness_values, c2):
    """Exploration phase when Q < threshold_food"""
    n_snakes, n_features = population.shape
    new_population = population.copy()
    
    for i in range(n_snakes):
        # Select random individual from same group (male/female)
        group_size = n_snakes // 2

        if i < group_size:  # Male group
            rand_idx = random.randint(0, group_size - 1)
        else:  # Female group  
            rand_idx = random.randint(group_size, n_snakes - 1)
        
        # Calculate A factor (Equation 7)
        if fitness_values[i] != 0:
            A = np.exp(fitness_values[rand_idx] / fitness_values[i])
        else:
            A = 1
        
        # Update position dimension by dimension (Equation 19)
        for j in range(n_features):
            random.seed(time.time() + i * n_features + j)
            update_prob = c2 * A * random.random()
            if random.random() < update_prob:
                new_population[i, j] = population[rand_idx, j]
    
    return new_population

def _exploitation_phase_high_temp(population, best_position, temp, Q, c3):
    """Exploitation phase when Temp > threshold_temp"""
    n_snakes, n_features = population.shape
    new_population = population.copy()
    
    for i in range(n_snakes):
        hamming_dist = _hamming_distance(best_position, population[i])
        S_value = _sigmoid_transfer(hamming_dist, Q)
        
        for j in range(n_features):
            if population[i, j] == best_position[j]:
                # Keep same dimension unchanged (Equation 24)
                new_population[i, j] = population[i, j]
            else:
                # Calculate H factor and update (Equation 25)
                H = c3 * temp * random.random() * S_value
                if random.random() < H:
                    new_population[i, j] = best_position[j]
    
    return new_population

def _exploitation_phase_low_temp(population, fitness_values, temp, Q, c3):
    """Exploitation phase when Temp < threshold_temp (Fight and Mating modes)"""
    n_snakes, n_features = population.shape
    new_population = population.copy()
    
    # Split into male and female groups
    n_males = n_snakes // 2
    n_females = n_snakes - n_males
    
    male_fitness = fitness_values[:n_males]
    female_fitness = fitness_values[n_males:]
    
    # Find best in each group
    best_male_idx = np.argmin(male_fitness)
    best_female_idx = np.argmin(female_fitness)
    best_male = population[best_male_idx]
    best_female = population[n_males + best_female_idx]
    
    # Update males
    for i in range(n_males):
        if random.random() < 0.5:  # Fight mode
            if male_fitness[i] != 0:
                F = np.exp(female_fitness[best_female_idx] / male_fitness[i])
            else:
                F = 1
            
            for j in range(n_features):
                update_prob = c3 * F * random.random() * Q
                if random.random() < update_prob:
                    new_population[i, j] = best_female[j]
        else:  # Mating mode
            partner_idx = random.randint(n_males, n_snakes - 1)
            
            if male_fitness[i] != 0:
                F = np.exp(fitness_values[partner_idx] / male_fitness[i])
            else:
                F = 1
            
            for j in range(n_features):
                update_prob = c3 * F * random.random() * Q
                if random.random() < update_prob:
                    new_population[i, j] = population[partner_idx, j]
    
    # Update females
    for i in range(n_females):
        actual_idx = i + n_males
        if random.random() < 0.5:  # Fight mode
            if female_fitness[i] != 0:
                F = np.exp(male_fitness[best_male_idx] / female_fitness[i])
            else:
                F = 1
            
            for j in range(n_features):
                update_prob = c3 * F * random.random() * Q
                if random.random() < update_prob:
                    new_population[actual_idx, j] = best_male[j]
        else:  # Mating mode
            partner_idx = random.randint(0, n_males - 1)
            
            if female_fitness[i] != 0:
                M = np.exp(male_fitness[partner_idx] / female_fitness[i])
            else:
                M = 1
            
            for j in range(n_features):
                update_prob = c3 * M * random.random() * Q
                if random.random() < update_prob:
                    new_population[actual_idx, j] = population[partner_idx, j]
    
    return new_population

def _apply_gaussian_mutation_transfer(population):
    """Apply Gaussian mutation transfer function to convert continuous to binary"""
    n_snakes, n_features = population.shape
    binary_population = np.zeros_like(population)
    
    for i in range(n_snakes):
        for j in range(n_features):
            binary_population[i, j] = _gaussian_mutation_transfer(population[i, j])
    
    return binary_population

def BSO(agents, fitAgent, trainX, testX, trainy, testy, dim, isNormalPop, curIter, MaxIter):
    
    """
    Binary Snake Optimizer adapted to match your PSO interface
    
    Parameters:
    - agents: Population of agents in your EOSA format
    - fitAgent: Not used, kept for interface compatibility
    - trainX, testX, trainy, testy: Training and testing data
    - dim: Number of features (dimensions)
    - isNormalPop: Should be False for EOSA format
    - curIter: Current iteration
    - MaxIter: Maximum iterations
    
    Returns:
    - Updated agents list
    """
    
    if agents == None and isNormalPop:
        dim = trainX.shape[1]  - 1
        print("size",dim)
        agents = []
        for g in range(10):
            features = []
            for j in range(dim):
                features.append(1 if random.random() > 0.5 else 0)
        
            agents.append(features)
    
    print(agents)

    # BSO Algorithm constants
    c1 = 0.5      # Equation 5: Q = c1 * exp((t-T)/T)
    c2 = 0.05     # Equation 6: exploration phase constant
    c3 = 2        # Equation 8: exploitation phase constant
    threshold_temp = 0.6   # Temperature threshold
    threshold_food = 0.25  # Food threshold
    
    popSize = len(agents)
    
    # Extract positions and fitness values
    population = np.zeros((popSize, dim))
    fitness_values = np.zeros(popSize)
    default_fitnesses =[]
    for i in range(popSize):
        if isNormalPop:
            population[i] = agents[i]
            # Calculate fitness if needed
            individual_fitness,individual_cost= EOSA_fitness(agents[i], trainX, testX, trainy, testy) 
            print("fitness for:",i,individual_fitness)
            default_fitnesses.append(individual_fitness)
            fitness_values[i], _ = EOSA_fitness(agents[i], trainX, testX, trainy, testy)
        else:
            population[i] = agents[i][EOSA_INDIVIDUAL][EOSA_ID_POS]
            fitness_values[i] = agents[i][EOSA_INDIVIDUAL][EOSA_ID_FIT]
    
    # Convert to binary space
    binary_population = np.zeros_like(population)
    for i in range(popSize):
        for j in range(dim):
            binary_population[i, j] = 1 if population[i, j] > 0.5 else 0
    
    print("Fitness before optimization",np.amin(default_fitnesses))

    # Find best solution
    best_idx = np.argmin(fitness_values)
    best_position = binary_population[best_idx].copy()
    
    # Update temperature and food quantity (Equations 4-5)
    t = curIter - 1  # Convert to 0-based indexing
    temp = np.exp(-t / MaxIter)
    Q = c1 * np.exp((t - MaxIter) / MaxIter)

    # Choose optimization phase based on food quantity
    if Q < threshold_food:
        print("type 1")
        # Exploration phase (Section 5.3)
        binary_population = _exploration_phase(binary_population, fitness_values, c2)
    else:
        # Exploitation phases
        if temp > threshold_temp:
            print("type 2")

            # High temperature exploitation (Section 5.4)
            binary_population = _exploitation_phase_high_temp(
                binary_population, best_position, temp, Q, c3
            )
        else:

            print("type 3")

            # Low temperature exploitation - Fight/Mating (Section 3)
            binary_population = _exploitation_phase_low_temp(
                binary_population, fitness_values, temp, Q, c3
            )
            
            # Replace worst individual with newborn
            worst_idx = np.argmax(fitness_values)
            for j in range(dim):
                binary_population[worst_idx, j] = 1 if random.random() > 0.5 else 0
    
    # Apply Gaussian mutation transfer function
    binary_population = _apply_gaussian_mutation_transfer(binary_population)
    
    # Convert back to continuous space and update agents
    popnew = []
    new_fitness_values = []

    for i in range(popSize):
        # Convert binary to continuous (for compatibility with your system)
        continuous_pos = binary_population[i].astype(float)
        
        if isNormalPop:
            popnew.append(continuous_pos)
            new_fitness,new_cost = EOSA_fitness(continuous_pos, trainX, testX, trainy, testy)
            print("new fitness",new_fitness)
            new_fitness_values.append(new_fitness)
        else:
            # Calculate new fitness
            fit, cost = EOSA_fitness(continuous_pos, trainX, testX, trainy, testy)
            
            # Maintain agent structure
            index, _ = agents[i]
            individual = index, [continuous_pos, fit]
            popnew.append(individual)
    
    print("after optimization",np.amin(new_fitness_values))
    return popnew



In [120]:
original_data_dir = './outputs/' +'checkpoints/original_text_data_features/breastEW_'
X = np.load(original_data_dir+"train_data.npy")
y = np.load(original_data_dir+"train_label.npy")
ty = np.load(original_data_dir+"eval_label.npy")
tX = np.load(original_data_dir+"eval_data.npy")
BSO(None,None,X,tX,y,ty,1,True,1,10);

size 29
[[0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0], [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1], [1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0], [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1], [1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1], [0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0], [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0]]
fitness for: 0 0.11124384236453198
fitness for: 1 0.10986453201970439
fitness for: 2 0.11089901477832